In [2]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

import pandas as pd
import torch 

cuda_available = torch.cuda.is_available()
device = 'cuda:0' if cuda_available else 'cpu'

torch.manual_seed(0)

data = pd.read_csv("data/dontpatronizeme_pcl.tsv",
                       sep="\t",
                       names=['par_id', 'art_id', 'keyword', 'country', 'text', 'label'],
                       skiprows=4)

data['label'] = data['label'].apply(lambda x: 0 if x in [0, 1] else 1)

trids = pd.read_csv('data/train_semeval_parids-labels.csv')
teids = pd.read_csv('data/dev_semeval_parids-labels.csv')

trids.par_id = trids.par_id
teids.par_id = teids.par_id

data

,par_id,art_id,keyword,country,text,label
0,1,@@24942188,hopeless,ph,"We 're living in times of absolute insanity , ...",0
1,2,@@21968160,migrant,gh,"In Libya today , there are countless number of...",0
2,3,@@16584954,immigrant,ie,White House press secretary Sean Spicer said t...,0
3,4,@@7811231,disabled,nz,Council customers only signs would be displaye...,0
4,5,@@1494111,refugee,ca,""" Just like we received migrants fleeing El Sa...",0
...,...,...,...,...,...,...
10464,10465,@@14297363,women,lk,Sri Lankan norms and culture inhibit women fro...,0
10465,10466,@@70091353,vulnerable,ph,He added that the AFP will continue to bank on...,0
10466,10467,@@20282330,in-need,ng,""" She has one huge platform , and information ...",1
10467,10468,@@16753236,hopeless,in,""" Anja Ringgren Loven I ca n't find a word to ...",1


In [3]:
# Rebuild training set

rows = [] # will contain par_id, label and text
for idx in range(len(trids)):
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

train_set = pd.DataFrame(rows)
# Split train into train and internal validation set 80:20

val_size = int(len(train_set) * 0.2)

train_set = train_set.sample(frac=1)
val_set = train_set.iloc[0:val_size].reset_index(drop=True).copy()
train_set = train_set.iloc[val_size:].reset_index(drop=True).copy()

In [4]:
# Rebuild test set

rows = [] # will contain par_id, label and text
for idx in range(len(teids)):
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

test_set = pd.DataFrame(rows)
test_set = test_set.sample(frac=1)

In [5]:
X = train_set['text']  
y = train_set['label']  

# Initialize the DummyClassifier
dummy_model = DummyClassifier(strategy='most_frequent', random_state=0)

dummy_model.fit(X, y)

# Make predictions
y_pred = dummy_model.predict(val_set['text'])

# Evaluate the model
print(classification_report(val_set['label'], y_pred))
print("Accuracy:", accuracy_score(val_set['label'], y_pred)) 

# 0.48 macro f1 on internal validation set.

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1528
           1       0.00      0.00      0.00       147

    accuracy                           0.91      1675
   macro avg       0.46      0.50      0.48      1675
weighted avg       0.83      0.91      0.87      1675

Accuracy: 0.9122388059701493


C:\Users\taow\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\taow\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\taow\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: U